## 

## Import libaries

In [33]:
import pandas as pd

from itertools import chain

## Import data

In [34]:
data = pd.read_json('./data/proposicoes_join_temas.json')
data.head()

,id,uri,siglaTipo,codTipo,numero,ano,ementa,tema
0,1198010,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2234,2023,Altera redação de dispositivos do artigo 4° da...,Defesa e Segurança
1,2345485,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,1,2023,Institui a Política Nacional de Longo Prazo.,Administração Pública
2,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Economia
3,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Finanças Públicas e Orçamento
4,2345625,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3,2023,Cria o Protocolo Não é Não de atendimento à mu...,Direito Penal e Processual Penal


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103902 entries, 0 to 103901
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         103902 non-null  int64 
 1   uri        103902 non-null  object
 2   siglaTipo  103902 non-null  object
 3   codTipo    103902 non-null  int64 
 4   numero     103902 non-null  int64 
 5   ano        103902 non-null  int64 
 6   ementa     103902 non-null  object
 7   tema       98133 non-null   object
dtypes: int64(4), object(4)
memory usage: 6.3+ MB


## Preprocesisng + EDA

Laws proposals

In [36]:
unique_laws_proposal = len(data['id'].unique())

print(f'{unique_laws_proposal} Unique laws')

71746 Unique laws


Show all the PL's unique themes

In [37]:
temas = data['tema'].unique().tolist()

print( len(temas), "unique themes\n", temas )

32 unique themes
 ['Defesa e Segurança', 'Administração Pública', 'Economia', 'Finanças Públicas e Orçamento', 'Direito Penal e Processual Penal', 'Direitos Humanos e Minorias', 'Indústria, Comércio e Serviços', 'Comunicações', 'Meio Ambiente e Desenvolvimento Sustentável', 'Saúde', 'Esporte e Lazer', 'Homenagens e Datas Comemorativas', 'Educação', 'Direito Constitucional', None, 'Relações Internacionais e Comércio Exterior', 'Viação, Transporte e Mobilidade', 'Trabalho e Emprego', 'Previdência e Assistência Social', 'Direito Civil e Processual Civil', 'Direito e Justiça', 'Agricultura, Pecuária, Pesca e Extrativismo', 'Turismo', 'Cidades e Desenvolvimento Urbano', 'Política, Partidos e Eleições', 'Direito e Defesa do Consumidor', 'Energia, Recursos Hídricos e Minerais', 'Estrutura Fundiária', 'Arte, Cultura e Religião', 'Ciência, Tecnologia e Inovação', 'Processo Legislativo e Atuação Parlamentar', 'Ciências Sociais e Humanas']


One-Hot-Encoding of the PL's themes

In [38]:
data.columns

Index(['id', 'uri', 'siglaTipo', 'codTipo', 'numero', 'ano', 'ementa', 'tema'], dtype='object')

In [39]:
data["count"] = 1

data = data.drop(
    columns=[
        "uri",
        "siglaTipo",
        "codTipo",
        "numero",
        "ano"
    ]
)

# pivot table
data_pivot = data.pivot_table(
    index=["id", "ementa"],
    columns="tema",
    values="count",
    aggfunc="sum",
    fill_value=0,
).reset_index()

data_pivot.head(3)

tema,id,ementa,Administração Pública,"Agricultura, Pecuária, Pesca e Extrativismo","Arte, Cultura e Religião",Cidades e Desenvolvimento Urbano,"Ciência, Tecnologia e Inovação",Ciências Sociais e Humanas,Comunicações,Defesa e Segurança,...,"Indústria, Comércio e Serviços",Meio Ambiente e Desenvolvimento Sustentável,"Política, Partidos e Eleições",Previdência e Assistência Social,Processo Legislativo e Atuação Parlamentar,Relações Internacionais e Comércio Exterior,Saúde,Trabalho e Emprego,Turismo,"Viação, Transporte e Mobilidade"
0,14919,"Dispõe sobre a Política Nacional de Salários, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,14920,"Modifica o art. 6º da Lei nº 9.424, de 24 de d...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14921,Dispõe sobre salário-família e dá outras provi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Show the 

In [40]:
data_pivot.drop(columns=['id', 'uri', 'codTipo', 'numero', 'ano', 'ementa']).sum().sort_values(ascending=False)

KeyError: "['uri', 'codTipo', 'numero', 'ano'] not found in axis"